In [1]:
from mglyph_ml.experiment.e1.experiment import ExperimentConfig, run_experiment


config = ExperimentConfig(
    task_name="Experiment 1.2.1",
    task_tag="exp-1.2.1",
    dataset_path="data/universal.mglyph",
    gap_start_x=40.0,
    gap_end_x=60.0,
    quick=True,
    seed=420,
    max_iterations=5,
    offline=True
)

In [ ]:
run_experiment(config)

ClearML Task: created new task id=offline-0b0e59c5ae5b4e01b1c5a500e686a5bb
ClearML running in offline mode, session stored in /home/xgaens00/.clearml/cache/offline/offline-0b0e59c5ae5b4e01b1c5a500e686a5bb
CLEARML new package available: UPGRADE to v2.1.3 is recommended!
Release Notes:
### New Features and Bug Fixes

- Fix GPU reporting for `NVIDIA_VISIBLE_DEVICES=void` (#1508)
- Fix default example parameters for sklearn joblib
- Add support for ClearML Apps Gateway static routes in Gradio binding

Using existing dataset: data/universal.mglyph
Device used for training: cuda


TypeError: GlyphDataset.__init__() missing 2 required positional arguments: 'gap_start_x' and 'gap_end_x'

ClearML running in offline mode, session stored in /home/xgaens00/.clearml/cache/offline/offline-0b0e59c5ae5b4e01b1c5a500e686a5bb
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start
